In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
from IPython.display import HTML
pd.set_option('display.max_rows', 500)

D:\JUPYTER\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [1]:
import fasttext

In [1]:
# Markes
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
from IPython.display import HTML
def get_markes():
    page = requests.get("https://rrr.lt/")
    soup = BeautifulSoup(page.content, 'html.parser')

    mark_list = soup.find(class_ ="cars_list clearfix")
#mark_list = soup.find_all(class_ = "cars_list__links")
    mark_list = mark_list.select(".cars_list__items a.cars_list__links")

    mark_title ={ list(mark_list[sd].children)[0][:-1]:mark_list[sd]["href"] for sd in range(len(mark_list))}

    markes = pd.DataFrame({
        "Markes": mark_title.items(),
    })
    mark_list = []
    for key, value in mark_title.items():
        mark_list.append(key)
    
    return mark_title, mark_list

def get_modeliai(x, marks):
    mark_title = marks
    page = requests.get(mark_title[x])
    soup = BeautifulSoup(page.content, 'html.parser')
    models_list = soup.find(class_="models_hold")
#print(models_list.prettify())
#model_title = [sd.get_text() for sd in models_list.select(".models .model-title")]
    models_list = models_list.select(".models .models__links")

    model_title ={ models_list[sd]['data-label']:models_list[sd]["href"] for sd in range(len(models_list))}
    
    model_list = []
    for key, value in model_title.items():
        model_list.append(key)
        
    return model_title, model_list

def get_category(x, models):
    model_title = models
    page = requests.get(model_title[x])
    soup = BeautifulSoup(page.content, 'html.parser')

    parts = soup.find(class_ = "parts")
    parts = parts.find_all(class_ = "parts__text")
# parts_link = soup.find_all(class_ = "parts_all-categories")
    parts_links = soup.select(".parts_all-categories a")
    parts_title ={list(parts[sd].children)[0][:-1].replace('\n                    ', '')+" >> "+str(list(parts[sd].children)[1].get_text()):parts_links[sd]["href"] for sd in range(len(parts))}

    raktai = []
    for key, value in parts_title.items():
        if re.search("Šioje kategorijoje dalių neturime", key):
            key = re.sub("Šioje kategorijoje dalių neturime", "(0)", key)
        raktai.append(key)
        
 # Read Note below
    
    return parts_title, raktai


def get_detales(x, marke, parts):
    parts_title = parts
    parts_spec = []
    parts_detales = []
    last = True
    next_ = parts_title[x]
    while(last):
        page = requests.get(next_)
        soup = BeautifulSoup(page.content, 'html.parser')
    
        page_last = soup.find(class_ = "pages__items last") 
        page_last = page_last.find(class_ = "pages__links")
    
        if page_last["class"] == ["pages__links"]:
            next_ = page_last["href"]
        else:
            last = False
        
        parts_details =soup.select(".products__text p")
        parts_specs = soup.select(".products .products__image")
        parts_price = soup.select(".products .products__price")
    
        parts_spec += [parts_specs[sd]["title"] + " " + parts_specs[sd]["href"] + " " +  parts_price[sd].get_text() for sd in range(len(parts_specs))]
    
        parts_detales += [parts_details[sd].get_text() for sd in range(len(parts_details))]
    
    a = 0
    ordered_list = []
#for i in parts_spec:
    #ordered_list.append(str(a) + ") " + i)
    #a += 1
#ordered_list

    item = []
    links = []
    kainos = []

    url =  "(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,})"
    for i in range(len(parts_spec)):
        x = re.search(".+?(?=https:)", parts_spec[i])
        y = re.search(url, parts_spec[i])
        item.append(x.group())
        links.append(y.group())
        e = parts_spec[i].split()[-1]
        k = parts_spec[i].split()[-2]
        price = e + k
        kainos.append(price)
    
    
    
    detales_info = [i for i in parts_detales if i.startswith(marke)]

    detales = pd.DataFrame({
        "Detalė": item,
        "Kaina": kainos, 
        "Info": detales_info,
        "Nuoroda": links,
    })
    detales_info_list = []
    
    for inp in range(len(detales)):
        pasirinkta_preke = detales.loc[inp, 'Nuoroda']

        page = requests.get(pasirinkta_preke)
        soup = BeautifulSoup(page.content, 'html.parser')

        detailed_info = soup.select(".product_desc .product_details")

        detailed_test = [sd.get_text() for sd in soup.select(".product_desc .product_details")]
        detailed_test = re.sub("\n", "|", detailed_test[1])
        detailed_test = detailed_test.split("|")
        
        del detailed_test[detailed_test == '']
        detales_info_dict = {}
        detales_info_list2 = []
        k = 0
        
        detales_info_list.append(detailed_test)
        
     #  for i in range(len(detailed_test)):
           #if k >= len(detailed_test)-1 :
                #reak
            #rint(detailed_test[k] + ": " + detailed_test[k+1])
            # += 2
        #etales_info_list.append(detales_info_list2)
        
   #detales["daugiau_info"] = detales_info_list
    
    
        #or i in range(len(detailed_test)):
        #   if k >= len(detailed_test)-1 :
        #       break
        #   print(detailed_test[k] + ": " + detailed_test[k+1])
        #   k += 2
        #etales_info_list.append(detales_info_list2)
        
    detales["daugiau_info"] = detales_info_list
    
    return detales

D:\JUPYTER\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [5]:
mark = get_markes()
model = get_modeliai("Audi", mark)

TypeError: tuple indices must be integers or slices, not str

In [2]:
# Markes
def get_markes():
    page = requests.get("https://rrr.lt/")
    soup = BeautifulSoup(page.content, 'html.parser')

    mark_list = soup.find(class_ ="cars_list clearfix")
#mark_list = soup.find_all(class_ = "cars_list__links")
    mark_list = mark_list.select(".cars_list__items a.cars_list__links")

    mark_title ={ list(mark_list[sd].children)[0][:-1]:mark_list[sd]["href"] for sd in range(len(mark_list))}

    markes = pd.DataFrame({
        "Markes": mark_title.items(),
    })
    mark_list = []
    for key, value in mark_title.items():
        mark_list.append(key)
    

    return

In [3]:
# rrr.lt
def get_modeliai(x):
    page = requests.get(mark_title[x])
    soup = BeautifulSoup(page.content, 'html.parser')
    models_list = soup.find(class_="models_hold")
#print(models_list.prettify())
#model_title = [sd.get_text() for sd in models_list.select(".models .model-title")]
    models_list = models_list.select(".models .models__links")

    model_title ={ models_list[sd]['data-label']:models_list[sd]["href"] for sd in range(len(models_list))}
    
    model_list = []
    for key, value in model_title.items():
        model_list.append(key)
        
    return model_title, model_list



In [4]:
# detales kategorijos

def get_category(x):
    page = requests.get(model_title[x])
    soup = BeautifulSoup(page.content, 'html.parser')

    parts = soup.find(class_ = "parts")
    parts = parts.find_all(class_ = "parts__text")
# parts_link = soup.find_all(class_ = "parts_all-categories")
    parts_links = soup.select(".parts_all-categories a")
    parts_title ={list(parts[sd].children)[0][:-1].replace('\n                    ', '')+" >> "+str(list(parts[sd].children)[1].get_text()):parts_links[sd]["href"] for sd in range(len(parts))}

    raktai = []
    for key, value in parts_title.items():
        if re.search("Šioje kategorijoje dalių neturime", key):
            key = re.sub("Šioje kategorijoje dalių neturime", "(0)", key)
        raktai.append(key)
        
 # Read Note below
    
    return parts_title, raktai
#pasirinkta_kategorija = re.search(pasirinkta_kategorija, parts_title.items())
#parts_list = parts.select("cars_list__items a.cars_list__links")


In [5]:
# Detales (full list)

def get_detales(x, marke):
    parts_spec = []
    parts_detales = []
    last = True
    next_ = parts_title[x]
    while(last):
        page = requests.get(next_)
        soup = BeautifulSoup(page.content, 'html.parser')
    
        page_last = soup.find(class_ = "pages__items last") 
        page_last = page_last.find(class_ = "pages__links")
    
        if page_last["class"] == ["pages__links"]:
            next_ = page_last["href"]
        else:
            last = False
        
        parts_details =soup.select(".products__text p")
        parts_specs = soup.select(".products .products__image")
        parts_price = soup.select(".products .products__price")
    
        parts_spec += [parts_specs[sd]["title"] + " " + parts_specs[sd]["href"] + " " +  parts_price[sd].get_text() for sd in range(len(parts_specs))]
    
        parts_detales += [parts_details[sd].get_text() for sd in range(len(parts_details))]
    
    a = 0
    ordered_list = []
#for i in parts_spec:
    #ordered_list.append(str(a) + ") " + i)
    #a += 1
#ordered_list

    item = []
    links = []
    kainos = []

    url =  "(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,})"
    for i in range(len(parts_spec)):
        x = re.search(".+?(?=https:)", parts_spec[i])
        y = re.search(url, parts_spec[i])
        item.append(x.group())
        links.append(y.group())
        e = parts_spec[i].split()[-1]
        k = parts_spec[i].split()[-2]
        price = e + k
        kainos.append(price)
    

    
    detales_info = [i for i in parts_detales if i.startswith(marke)]

    detales = pd.DataFrame({
        "Detalė": item,
        "Kaina": kainos, 
        "Info": detales_info,
        "Nuoroda": links,
    })
    return detales

['Brilliance BS6 2004, 2000cm3, Benzinas, Mechaninė',
 'Brilliance BS6 2004, 2000cm3, Benzinas, Mechaninė',
 'Brilliance BS6 2004, 2000cm3, Benzinas, Mechaninė',
 'Brilliance BS6 2004, 2000cm3, Benzinas, Mechaninė',
 'Brilliance BS6 2004, Nenurodyta, Mechaninė',
 'Brilliance BS6 2004, 2000cm3, Benzinas, Mechaninė',
 'Brilliance BS6 2004, 2000cm3, Benzinas, Mechaninė']

In [54]:
# SEARCH
a = input('Ka norite rasti?') # b 
detales[detales['Detalė'].str.contains(a)]

KeyboardInterrupt: Interrupted by user

In [101]:
inp = int(input("Apie kuria preke norite daugiau sužinoti?"))

pasirinkta_preke = detales.loc[inp, 'Nuoroda']

page = requests.get(pasirinkta_preke)
soup = BeautifulSoup(page.content, 'html.parser')

detailed_info = soup.select(".product_desc .product_details")

detailed_test = [sd.get_text() for sd in soup.select(".product_desc .product_details")]
detailed_test = re.sub("\n", "|", detailed_test[1])
detailed_test = detailed_test.split("|")
del detailed_test[detailed_test == '']

k = 0
for i in range(len(detailed_test)):
    if k >= len(detailed_test)-1 :
        break
    print(detailed_test[k] + ": " + detailed_test[k+1])
    k += 2
    
print("Daugiau informacijos: " + detales.loc[inp, 'Nuoroda'])

Apie kuria preke norite daugiau sužinoti?49
Gamintojas: Honda
Serija: Accord
Modelis: Accord
Gamybos metai: 2008-2016
Metai: 2009
Kėbulo tipas: Sedanas
Vairo padėtis: Dešinėje
Varomieji ratai: Priekiniai
Kuro tipas: Dyzelinas
Variklio tūris, cm3: 2200
Variklio galia, kW: 110
Greičio dėžė: Mechaninė
Spalva: Pilka
Daugiau informacijos: https://rrr.lt/autodalis/val97633-honda-accord-priekine-sake


In [ ]:
import tkinter as tk
from tkinter import filedialog, messagebox, ttk

import pandas as pd

def mark_changed(event):
    global model_title
    model_title, x = get_modeliai(Combo_mark.get())
    Combo_model.config(value = x)
    print(model_title)
    

def model_changed(event):
    global parts_title
    parts_title, x = get_category(Combo_model.get())
    
    for i in range(len(x)):
        listbox.delete(i)
    for i in range(len(x)):
        listbox.insert(i, x[i])

def category_changed(event):
    global detales
    print(listbox.curselection())

root = tk.Tk()
root.geometry("500x500")
root.pack_propagate(False)
root.resizable(0, 0)

frame1 = tk.LabelFrame(root, text = "Detales")
frame1.place(height=250, width=500)

file_frame = tk.LabelFrame(root, text = "Ieskoti")
file_frame.place(height = 100, width = 250, rely=0.50,relx = 0.50)

##
tv1 = ttk.Treeview(frame1)
tv1.place(relheight=1,relwidth=1)

search = tk.Entry(file_frame, width = 15)
search.place(rely=0.40, relx = 0.25)

treescrolly = tk.Scrollbar(frame1, orient="vertical", command=tv1.yview)
treescrollx = tk.Scrollbar(frame1, orient="horizontal", command=tv1.xview)
tv1.configure(xscrollcommand=treescrollx.set, yscrollcommand=treescrolly.set)
treescrollx.pack(side="bottom", fill = "x")
treescrolly.pack(side="right", fill= "y")

button1 = tk.Button(root, text = "Visi prekės",height = 3, width = 20, command=lambda: Load_data())
button1.place(rely=0.80, relx=0.60)

button2 = tk.Button(file_frame, text = "Paieška", command=lambda: Search())
button2.place(rely=0.35, relx=0.65)
model_list = []
marke = tk.StringVar()
model = tk.StringVar()

Combo_mark = ttk.Combobox(root, values = mark_list)
Combo_mark.set("Markė")
Combo_mark.place(relx = 0.05, rely = 0.55)
## MARKE
Combo_mark.bind('<<ComboboxSelected>>',  mark_changed)


Combo_model = ttk.Combobox(root, values = [])
Combo_model.set("Modelis")
Combo_model.place(relx = 0.05, rely = 0.60)
Combo_model.bind('<<ComboboxSelected>>', model_changed)
## Model

listbox = tk.Listbox(root, width = 35, selectmode = "SINGLE")
listbox.place(relx = 0.05, rely = 0.65)
listbox.bind('<<ListboxSelect>>', category_changed)


def Load_data():
    clear_data()
    
    detales = get_detales(listbox.get("anchor"), Combo_mark.get())
    
    tv1["column"] = list(detales.columns)
    tv1["show"] = "headings"
    for column in tv1["column"]:
        tv1.heading(column, text=column)
    
    df_rows = detales.to_numpy().tolist()
    for row in df_rows:
        tv1.insert("", "end", values=row)
    return None

def Search():
    clear_data()
    a = str(search.get())
    d1 = detales[detales['Detalė'].str.contains(a)]
        
    tv1["column"] = list(d1.columns)
    tv1["show"] = "headings"
    for column in tv1["column"]:
        tv1.heading(column, text=column)
    
    df_rows = d1.to_numpy().tolist()
    for row in df_rows:
        tv1.insert("", "end", values=row)
    return None

def clear_data():
    tv1.delete(*tv1.get_children())


root.mainloop()

{'1500 2500 (1962-1977)': 'https://rrr.lt/naudotos-autodalys/bmw/1500-2500-1962-1977', '501 (1952-1962)': 'https://rrr.lt/naudotos-autodalys/bmw/501-1952-1962', 'i3 (2013--)': 'https://rrr.lt/naudotos-autodalys/bmw/i3-2013', 'i8 (2014-)': 'https://rrr.lt/naudotos-autodalys/bmw/i8-2014', 'Isetta (1955-1962)': 'https://rrr.lt/naudotos-autodalys/bmw/isetta-1955-1962', 'M1 E26 (1978-1981)': 'https://rrr.lt/naudotos-autodalys/bmw/m1-e26-1978-1981', 'M2 F87 (2015-)': 'https://rrr.lt/naudotos-autodalys/bmw/m2-f87-2015-2825', '1 E81 E87 (2004-2011)': 'https://rrr.lt/naudotos-autodalys/bmw/1-e81-e87-2004-2011', '1 E82 E88 (2007-2014)': 'https://rrr.lt/naudotos-autodalys/bmw/1-e82-e88-2007-2014', '1 F20 F21 (2011-2019)': 'https://rrr.lt/naudotos-autodalys/bmw/1-f20-f21-2011-2019', '1 F40 (2019-)': 'https://rrr.lt/naudotos-autodalys/bmw/1-f40-2019', '2 114 E6 E10 E20 (1966-1975)': 'https://rrr.lt/naudotos-autodalys/bmw/2-114-e6-e10-e20-1966-1975', '2 F22 F23 (2013-2017)': 'https://rrr.lt/naudotos

In [66]:
#detales = detales[detales['Detalė'].str.contains("Pagrindinis")]
#detales
parts_title

{'Apšvietimo sistema >> (83)': 'https://rrr.lt/naudotos-autodalys/bmw/i3-2013/apsvietimo-sistema?man_id=1&cmc=1093&cm=1959',
 'Degalų mišinio sistema >> (107)': 'https://rrr.lt/naudotos-autodalys/bmw/i3-2013/degalu-misinio-sistema?man_id=1&cmc=1093&cm=1959',
 'Dujų išmetimo sistema >> (32)': 'https://rrr.lt/naudotos-autodalys/bmw/i3-2013/duju-ismetimo-sistema?man_id=1&cmc=1093&cm=1959',
 'Durys >> (168)': 'https://rrr.lt/naudotos-autodalys/bmw/i3-2013/durys?man_id=1&cmc=1093&cm=1959',
 'Galinė ašis >> (61)': 'https://rrr.lt/naudotos-autodalys/bmw/i3-2013/galine-asis?man_id=1&cmc=1093&cm=1959',
 'Galinės išorės detalės >> (187)': 'https://rrr.lt/naudotos-autodalys/bmw/i3-2013/galines-isores-detales?man_id=1&cmc=1093&cm=1959',
 'Kėbulas/ kėbulo dalys/ kablys >> (424)': 'https://rrr.lt/naudotos-autodalys/bmw/i3-2013/kebulas-kebulo-dalys-kablys?man_id=1&cmc=1093&cm=1959',
 'Kitos detalės >> (236)': 'https://rrr.lt/naudotos-autodalys/bmw/i3-2013/kitos-detales?man_id=1&cmc=1093&cm=1959',
 'O

In [4]:
x = "Apšvietimo sistema >> (4)"
a = re.compile("[^>]*")

temp = re.search(a, x)
temp = temp.group()
temp = temp[:-1]


'Apšvietimo sistema'

In [74]:
import unittest
from selenium import webdriver
import page

class PythonOrgSearch(unittest.TestCase):
    def setUp(self):
        self.driver = webdriver.Firefox()
        self.driver.get("http://www.python.org")
    def test_search_in_python_org(self):

#Load the main page. In this case the home page of Python.org.
        main_page = page.MainPage(self.driver)
#Checks if the word "Python" is in title
        assert main_page.is_title_matches(), "python.org title doesn't match."
#Sets the text of search textbox to "pycon"
        main_page.search_text_element = "pycon"
        main_page.click_go_button()
        search_results_page = page.SearchResultsPage(self.driver)
#Verifies that the results page is not empty
        assert search_results_page.is_results_found(), "No results found."
def tearDown(self):
        self.driver.close()
if __name__ == "__main__":
        unittest.main()

ModuleNotFoundError: No module named 'page'

In [3]:
#import tensorflow as tf
import tensorflow_hub as hub
#embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/1")



AttributeError: module 'tensorflow.tools.docs.doc_controls' has no attribute 'inheritable_header'

In [2]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 

Instructions for updating:
non-resource variables are not supported in the long term
